In [1]:
import pandas as pd
import datetime as dt
import requests
import matplotlib.pyplot as plt

In [2]:
calender = pd.read_csv('Calendar-Table 1.csv')
location = pd.read_csv('Locations-Table 1.csv')
products = pd.read_csv('Products-Table 1.csv')
sales = pd.read_csv('Sales-Table 1.csv')
states = pd.read_csv('States-Table 1.csv')

## Data cleaning

In [3]:
#calculate the revenue_USD
sales['Unit Price'] = sales['Unit Price'].map(lambda x: x.strip('US$')).astype(float)
sales['revenue_USD'] = sales['Unit Sales'] * sales['Unit Price']

#format date
sales['date_formatted'] = pd.TimedeltaIndex(sales['Date'], unit='d') + dt.datetime(1900,1,1)

#add quarter info
sales['Quarter'] = sales['date_formatted'].dt.to_period('Q')

#drop nan column
sales.drop(columns = 'Unnamed: 5', inplace = True)

#get USD/CNY exchange rate through API
url = 'https://api.exchangerate-api.com/v4/latest/USD'
response = requests.get(url).json()
USD_to_CNY_rate = response['rates']['CNY']

#calculate revenue_CNY
sales['revenue_CNY'] = round(sales['revenue_USD'] * USD_to_CNY_rate , 2)

In [4]:
#formate date
calender['date_formatted'] = pd.TimedeltaIndex(calender['Date'], unit = 'd') + dt.datetime(1900,1,1)
calender = calender[['date_formatted']]

In [5]:
#add state code
US_states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
states['state_code'] = states['State'].map(lambda x: US_states.index(x))
states.drop(columns =['Unnamed: 4'], inplace = True)

In [6]:
location.drop(columns = ['Unnamed: 4','Unnamed: 5'],inplace = True)

In [7]:
products.drop(columns = ['Unnamed: 5'], inplace = True)

## Calculate revenue per day

In [8]:
sales.groupby(['date_formatted']).sum()[['revenue_USD']]

,revenue_USD
date_formatted,
2014-02-02,7389817.40
2014-03-02,7290746.75
2014-04-02,7104319.25
2014-05-02,7068238.55
2014-06-02,6926530.10
2014-07-02,7391033.30
2014-08-02,7608650.55
2014-09-02,7639620.65
2014-10-02,7647030.60


## Dataframe intergration

In [9]:
df = sales.merge(location, on = ['Store'], how = 'left')
df = df.merge(products, on = ['Product ID'])
df = df. merge(states, on = ['State'])

In [10]:
#import the csv
df.to_csv('df.csv')